In [1]:
import cobra
import os
import pandas as pd
import numpy as np
import urllib
from cobrame.util import building, mu, me_model_interface

## Metabolites

met_output_file = 'metabolites.txt'
m_model_file = './iCW773R.json'

m_model = cobra.io.load_json_model(m_model_file)

/home/tanyingqi/anaconda3/envs/bacillusme/lib/python3.6/site-packages/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/tanyingqi/anaconda3/envs/bacillusme/lib/python3.6/site-packages/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


In [2]:
def duplicates(lst, item, counter):
    #找到列表中除指定位置外的所有重复项的索引
    return [i for i, x in enumerate(lst) if x == item and i != counter]

def duplicate_indices(lst, item):
    #找到列表中所有与指定元素匹配的索引
    return [i for i, x in enumerate(lst) if x == item]

def convert_compartments(compartment_list):
    #将代谢物隔室的简写形式转换为完整形式
    compartment_ids = ['c','e','p']
    compartment_equivalence = ['Cytosol','Extra-organism','Periplasm']
    
    for c in range(0,len(compartment_list)):
        if compartment_list[c] in compartment_ids:
            ind = compartment_ids.index(compartment_list[c])
            compartment_list[c] = compartment_equivalence[ind]
            
    return compartment_list

def convert_compartment(compartment_list):
    #将单个隔室简写形式转换为完整形式
    compartment_ids = ['c','e','p']
    compartment_equivalence = ['Cytosol','Extra-organism','Periplasm']
    if compartment_list in compartment_ids:
        ind = compartment_ids.index(compartment_list)
        compartment_list = compartment_equivalence[ind]
            
    return compartment_list

## 1. metabolites.txt

In [3]:
def generate_metfile(m_model,filename):
    file = open(filename,'w')
    
    metabolite_list = [None]*len(m_model.metabolites)
    compartment_list = [None]*len(m_model.metabolites)
    
    for m in range(0,len(m_model.metabolites)):
        ID = m_model.metabolites[m].id
        compartment_list[m] = ID[len(ID)-1]
        metabolite_list[m] = ID[:len(ID)-2]
        
    compartment_list = convert_compartments(compartment_list)
    
    other_mets_index = [];
    for c in range(0,len(m_model.metabolites)):
        if len(duplicates(other_mets_index,c,-1)) == 0:
            met = metabolite_list[c]
            name = m_model.metabolites[c].name
            formula = m_model.metabolites[c].formula

            other_mets_index = duplicates(metabolite_list,met,c)
            
            if formula is None:
                formula = 'no-formula'
            
            string = met + '\t' + name + '\t' + formula + '\t' + compartment_list[c]
            for other_mets in other_mets_index:
                string = string + ' AND ' + compartment_list[other_mets]
            
            string = string + '\n'
            file.write(string)
    file.close()

In [4]:
generate_metfile(m_model,'metabolites.txt')

## reaction_matrix.txt

In [5]:
def generate_reactionmatrix_file(m_model,filename):
    file = open(filename,'w')
    
    reaction_matrix_list = []
    
    for rxn in m_model.reactions:
        for metabolite in rxn.metabolites:
            compartment = metabolite.compartment
            compartment = convert_compartment(compartment)
            
            met = metabolite.id
            met = met[:len(met)-2]
            
            coefficient = rxn.get_coefficient(metabolite)
            
            string = rxn.id + '\t' + met + '\t' + compartment + '\t' + str(coefficient) + '\n'
            
            file.write(string)

    file.close()
    
    
    

In [6]:
generate_reactionmatrix_file(m_model,'reaction_matrix.txt')

In [7]:
import pandas
m_to_me_df = pandas.read_csv('m_to_me_mets.csv', index_col=0)
m_to_me_dict = m_to_me_df.to_dict()['me_name']
m_to_me_dict

{'ACP_c': 'BSU15920-MONOMER_mod_pan4p',
 'apoACP_c': 'BSU15920-MONOMER',
 'fmettrna_c': 'eliminate',
 'glutrna_c': 'eliminate',
 'mettrna_c': 'eliminate',
 'trnaglu_c': 'eliminate',
 'trnamet_c': 'eliminate',
 'trdox_c': 'BSU28500-MONOMER_mod_Oxidized',
 'trdrd_c': 'BSU28500-MONOMER'}

In [8]:
reaction_matrix_file = 'reaction_matrix.txt'
with open(reaction_matrix_file,'U') as f:
    newText = f.read()
    for mets_to_replace in m_to_me_dict.keys():
        if m_to_me_dict[mets_to_replace] == 'eliminate':
            continue
        met_string = '\t' + mets_to_replace.split('_')[0] + '\t'
        replace_string = '\t' + m_to_me_dict[mets_to_replace] + '\t'
        newText = newText.replace(met_string,replace_string)
with open(reaction_matrix_file,'w') as f:
    f.write(newText)

/home/tanyingqi/anaconda3/envs/bacillusme/lib/python3.6/site-packages/ipykernel_launcher.py:2 DeprecationWarning: 'U' mode is deprecated


## reactions.txt

In [11]:
# def generate_reactions_file0(m_model,filename):
#     file = open(filename,'w')
    
#     string = '#name' + '\t' + 'description' +  '\t' + 'is_reversible' + '\t' + 'data_source' + '\t' + 'is_spontaneous' +  '\n'
    
#     file.write(string)
#     reaction_matrix_list = []
    
#     for rxn in m_model.reactions:
#         rev = rxn.reversibility
#         name = rxn.name
#         name = name.replace(' ','_')
        
#         if rev == True:
#             reversibility = 1
#         else:
#             reversibility = 0
        
#         if 'diffusion' in rxn.name:
#             is_spontaneous = 1
#         else:
#             is_spontaneous = 0
        
#         string = rxn.id + '\t' + name + '\t' + str(reversibility) + '\t' + m_model.id + '\t' + str(is_spontaneous) + '\n'
#         file.write(string)

#     file.close()
    
# generate_reactions_file(m_model,'reactions.txt')

In [13]:
# 修改后
def generate_reactions_file(m_model, filename):
    with open(filename, 'w') as file:
        header = '#name\t' + 'description\t' + 'is_reversible\t' + 'data_source\t' + 'is_spontaneous\n'
        file.write(header)

        for rxn in m_model.reactions:
            rev = rxn.reversibility
            name = rxn.name.replace(' ', '_')
            reversibility = 1 if rev else 0
            is_spontaneous = 1 if 'diffusion' in rxn.name else 0

            # 检查 rxn.id 和 m_model.id 是否为 None，如果是，则替换为空字符串
            rxn_id = rxn.id if rxn.id is not None else ''
            m_model_id = m_model.id if m_model.id is not None else ''

            # 构建字符串并写入文件
            string = f'{rxn_id}\t{name}\t{reversibility}\t{m_model_id}\t{is_spontaneous}\n'
            file.write(string)

# 调用函数生成 reactions.txt 文件
generate_reactions_file(m_model, 'reactions.txt')